In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import matplotlib.image as mpimg
%matplotlib inline
import math

# Import API key
from config import gapi

In [2]:
# Loan CSV file generated from WeatherPy Folder
weather_data_to_load = "../WeatherPy/output_data/cities.csv"
weather_data = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_data.dropna()
dropna_weather_data.head()

,city_id,city,Cloudiness,Humidity,Latitude,Longitude,Country,Max Temp,Wind Speed
0,0,harindanga,40,94,22.04,88.32,IN,84.20,4.70
1,1,carnarvon,75,100,-24.87,113.63,AU,62.60,10.29
2,2,avarua,0,73,-21.21,-159.78,CK,71.60,9.17
3,3,tamale,100,95,9.40,-0.84,GH,73.42,7.52
4,4,krasnyy,100,87,54.57,31.43,RU,53.67,6.26


#### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=gapi)

# Locations
locations = dropna_weather_data[['Latitude', 'Longitude']]
humidity = dropna_weather_data['Humidity'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

#### Narrow down DataFrame to ideal weather conditions

In [4]:
# vacation spots with perfect weather 

# zero cloudiness
vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness"] == 0]

# temp between 70 and 80 
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp"] > 70]
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp"] < 80]

# wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed"] < 10]

# humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity"] < 60]

# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,city_id,city,Cloudiness,Humidity,Latitude,Longitude,Country,Max Temp,Wind Speed
0,122,loralai,0,52,30.37,68.60,PK,78.73,5.30
1,126,sao desiderio,0,36,-12.36,-44.97,BR,76.95,5.55
2,143,bayir,0,50,37.27,28.22,TR,77.00,4.70
3,153,itapuranga,0,36,-15.56,-49.95,BR,74.70,0.58
4,361,urumqi,0,36,43.80,87.60,CN,78.80,8.95
5,525,kanashevo,0,53,55.22,62.06,RU,75.20,6.71


In [5]:
vaca_locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

vaca_humidity = indexed_perfect_vacation["Humidity"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=10.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

# vaca_fig = plt.imshow(vaca_fig)

Figure(layout=FigureLayout(height='420px'))

##### Hotel Map

In [6]:
# hotel variable 
hotels = []
params = {
    'radius':5000, 
    'types': 'lodging', 
    'key': gapi
    
}
for index, row in indexed_perfect_vacation. iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    params['location']=f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    # convert to json
    name_address = name_address.json()
    print(name_address)
    # Grab the first hotel from the results and store the name
    try:
        indexed_perfect_vacation.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

indexed_perfect_vacation

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAHnhk7I9DnrHaVxfzQ7ulX4INy5NzluMolJ3nadNA0kDLLkBynG7J5REDn3R9G5Q1dAPl4nW17urIx033dfLr4hdDUT4c2Jo1S1z1GJB--UZQAvU_P1FhMALS38GCq2Tb4WjbdIueRt_YIMgXBdAdoHqQ94VK-bUxDTpKFGtd4IzR2GhnLeXYuBCKBE7bntl7kqwkL40dqXjij5fLIELw7qOwIisqBajKn22bU2iqD2L14n0FDjGIictpp8TDAyj_do62dCrb4FL0kexv9mZpQnAyRt18oRoYwsEYHEWNP_J6CPFtr_SCmXkK5PCoF2E1mUaE9wsLQWLRm2rcDfunNseKTqHg-_8hKuvL3C3dXXlpsC0uqZoMJV228Oc5UJPKRIQw9rmN2Otyuvw__DJkME0ORoUyK09WIqg2ew91F0tL99--c0TRWU', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 30.3709727, 'lng': 68.5995615}, 'viewport': {'northeast': {'lat': 30.3723128802915, 'lng': 68.6009050802915}, 'southwest': {'lat': 30.3696149197085, 'lng': 68.59820711970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'c1c80d9c61290ec1f66efe1da988b0b1f790ea0b', 'name': 'City Hotel. Loralai. Blochistan', 'photos': [{'height': 3264, 'html_attributions': ['<a href="https://maps.goog

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -12.3566101, 'lng': -44.9777094}, 'viewport': {'northeast': {'lat': -12.3553067697085, 'lng': -44.9763200697085}, 'southwest': {'lat': -12.3580047302915, 'lng': -44.9790180302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '155719e61158679d3275023ff2b4adb6bf76a679', 'name': 'Pousada do Luizão', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJ_zKqociAXwcRfe36GvPj8ec', 'plus_code': {'compound_code': 'J2VC+9W São Desidério, State of Bahia, Brazil', 'global_code': '58VQJ2VC+9W'}, 'rating': 5, 'reference': 'ChIJ_zKqociAXwcRfe36GvPj8ec', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 2, 'vicinity': 'São Desidério'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -12.3658143, 'lng': -44.9783662}, 'viewport': {'northeast': {'lat': -12.3644539697085, 'lng': -44.97702871

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAJiMCF29M7464rA_V7vcet6ZtCQiU4NTUilRk-WDBCDVvxvsn3N01LvKjp3OCfKy_HQJHngltoVUlpYw0QaP-MwViDuYuejy52PobisCH6ib-YKcHI-SiYSC1ZiWrveSr_cVBPKfyo2ylYgN7EuFYO64vxEY9BCcq6lBOUlpKwN0_BVTUN4uQ9SR6ukODm1KaMek1C3Wq6mr24DVg2-iv8yHWUCcyeoxikqgzWkZMqLY2J4JMvl0NAmiYPSOn3qsH42Aw25RlFKrQTmEJKBFDl03HYtdTG3vKuXvtoUtdLlql9ypOEaF4UNYeGUvNj4rBa6hDblJ6aQXcJoPtvBb-0n2aN4XYzuRo_ZO5Ri1U77S4bmnxnyhe7LnRFkX3j8ftBIQ8CdcZaydbXeqLwIkmbNw8hoUIqVtJ7HYY7h185VHdrhopOTDDsw', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 43.7952, 'lng': 87.58019999999999}, 'viewport': {'northeast': {'lat': 43.7966215302915, 'lng': 87.5815579802915}, 'southwest': {'lat': 43.7939235697085, 'lng': 87.57886001970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '33230094cef2635a3b3d4de67959975ac5327957', 'name': 'Hua Ling Grand Hotel Urumqi', 'photos': [{'height': 4032, 'html_attributions': ['<a href="https://maps.goog

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 55.2033056, 'lng': 62.02445839999999}, 'viewport': {'northeast': {'lat': 55.20460823029149, 'lng': 62.02596023029151}, 'southwest': {'lat': 55.2019102697085, 'lng': 62.0232622697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '6f651cd02fdc3162cb17044e060fce1b1b14eb59', 'name': 'Klim', 'photos': [{'height': 4032, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102792909987643355747">Пиксаева Елена</a>'], 'photo_reference': 'CmRaAAAA5jrp2mKSzRSB_janoWDHj3VmwM3TEqN-lZkvwsnW3TIwanzshc4Dk6IHqzDNzHgbMQ5WQ5zf5o78oC9xeBjiL5z_CfDEul4hy-Lwt787M61xgEaaFTg-vol7r2CdfRl8EhCkO8LQf5EC_Jl3d46MVghAGhQ-0vNOhmZ-XGuONck47eoV6rdEWg', 'width': 2268}], 'place_id': 'ChIJsQ4rPgf-xUMR3JhMYXw-5_s', 'plus_code': {'compound_code': '623F+8Q Kanashevo, Chelyabinsk Oblast, Russia', 'global_code': '9J74623F+8Q'}, 'rating': 3.6, 'reference': 'ChIJsQ4rPgf

,city_id,city,Cloudiness,Humidity,Latitude,Longitude,Country,Max Temp,Wind Speed,Hotel Name
0,122,loralai,0,52,30.37,68.60,PK,78.73,5.30,City Hotel. Loralai. Blochistan
1,126,sao desiderio,0,36,-12.36,-44.97,BR,76.95,5.55,Pousada do Luizão
2,143,bayir,0,50,37.27,28.22,TR,77.00,4.70,Buban Otel
3,153,itapuranga,0,36,-15.56,-49.95,BR,74.70,0.58,Terraço Hotel
4,361,urumqi,0,36,43.80,87.60,CN,78.80,8.95,Hua Ling Grand Hotel Urumqi
5,525,kanashevo,0,53,55.22,62.06,RU,75.20,6.71,Klim


In [7]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

In [8]:
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
vaca_fig.add_layer(markers)
vaca_fig

Figure(layout=FigureLayout(height='420px'))